In [26]:
# pip install vk-api
import vk_api
import pandas as pd
from datetime import timedelta
from dotenv import load_dotenv
import os

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")


In [34]:
def get_vk_posts_data(token, user_id, count=100):
    vk_session = vk_api.VkApi(token=token)
    vk = vk_session.get_api()

    data = []
    total_count = count
    offset = 0

    while total_count > 0:
        batch_count = min(total_count, 100)  
        posts = vk.wall.get(owner_id=user_id, count=batch_count, offset=offset)['items']

        if not posts:
            break

        for post in posts:
            date = pd.to_datetime(post['date'], unit='s')
            likes = post['likes']['count']
            data.append([date, likes])

        total_count -= len(posts)
        offset += len(posts)

    df = pd.DataFrame(data, columns=['Date', 'Likes'])
    # df = df.sort_values(by='Date')
    df = df.sort_values(by='Date')
    df['Hour'] = df['Date'].dt.hour
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Interval'] = df['Date'].diff().dt.total_seconds() / 86400
    df['Interval'] = df['Interval'].abs()
    return df

ACCESS_TOKEN='vk1.a.v-7O0SD8tZIPj6VB64aPQtZ9q9MANBa8GY7i0zmu8pYS0gZ4kaWpdj5XbZRPnQ_h0Jp_YlrWiaG0VVsoZEDCbnEnjzwUogAr8jPTsXLk_KQShK_T6224Yssmny2SIp8gHK2ZaOi5IZ8WhaoOxbDUgt-yKZ4ToMJVI-m5IQ2kqyBm5Nsqsw82oV3o_nBo3N39kRioP9A_9UDcICLmXOn3iA'
user_id = '236263371'
df = get_vk_posts_data(ACCESS_TOKEN, user_id)
print(df)
print(df.isnull().sum())
df = df.dropna(subset=['Interval'])
print(df.isnull().sum())

                  Date  Likes  Hour  DayOfWeek    Interval
83 2013-12-27 07:40:19      9     7          4         NaN
82 2013-12-27 08:53:43      2     8          4    0.050972
81 2013-12-27 10:30:05     22    10          4    0.066921
80 2013-12-29 07:35:29      5     7          6    1.878750
79 2013-12-29 08:22:23      3     8          6    0.032569
..                 ...    ...   ...        ...         ...
4  2018-06-24 08:53:06      2     8          6  155.474618
3  2018-07-02 14:27:43     43    14          0    8.232373
2  2019-03-01 17:24:20      5    17          4  242.122650
1  2019-03-29 20:44:28      1    20          4   28.138981
0  2019-11-23 14:53:19     15    14          5  238.756146

[84 rows x 5 columns]
Date         0
Likes        0
Hour         0
DayOfWeek    0
Interval     1
dtype: int64
Date         0
Likes        0
Hour         0
DayOfWeek    0
Interval     0
dtype: int64


In [30]:
import statsmodels.api as sm
import numpy as np

def analyze_factors(df):
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
    
    df['PostFrequency'] = df.groupby(df['Date'].dt.date)['Likes'].transform('count')


    df['LogLikes'] = np.log1p(df['Likes'])
    
    # df = df.dropna(subset=['Interval'])

    # y = df['Likes']
    # x = df[['Hour', 'DayOfWeek', 'Interval']]
    X = df[['Hour', 'DayOfWeek', 'Interval', 'IsWeekend', 'PostFrequency']]
    y = df['LogLikes']
    
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)
    print(model.summary())

analyze_factors(df)
print(df.isnull().sum())

                            OLS Regression Results                            
Dep. Variable:               LogLikes   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     2.689
Date:                Fri, 20 Sep 2024   Prob (F-statistic):             0.0271
Time:                        22:36:40   Log-Likelihood:                -113.20
No. Observations:                  83   AIC:                             238.4
Df Residuals:                      77   BIC:                             252.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.8402      0.483      5.881

In [ ]:
Были добавлены новые признаки, такие как 'Interval', 'IsWeekend', 'PostFrequency', 'LogLikes'(для логарифмическая трансформации 
для сглаживания выбросов) для более точного анализа

R² = 0.139: Это означает, что модель объясняет только 13.9% изменений в количестве лайков. В принципе, 
это низкий результат, могут быть другие важные факторы, которые не учтены.
Статистическая значимость:
Выходные дни (IsWeekend): p-значение = 0.043. Это значит, что посты, опубликованные в выходные, 
получают больше лайков, и этот фактор важен.
Частота постов (PostFrequency): p-значение = 0.060. Этот фактор почти значим, указывая на то, 
что слишком частые посты могут уменьшать количество лайков.
День недели (DayOfWeek), интервал между постами (Interval), и время суток (Hour): 
Эти факторы не влияют на количество лайков существенно.

Коэффициенты:
Выходные (0.8212): Посты, опубликованные в выходные, получают больше лайков.
Частота постов (-0.2792): Если постить слишком часто, количество лайков уменьшается.
День недели, время и интервал: Эти факторы оказывают незначительное влияние.